In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tata20y/TATAMOTORS-20Y.csv
/kaggle/input/initialclosingdata/original.csv


In [2]:
tata_df = pd.read_csv("../input/tata20y/TATAMOTORS-20Y.csv")
df = tata_df["Close"]
df

0        17.132215
1        18.032904
2        19.518078
3        20.073820
4        19.757624
           ...    
5094    315.299988
5095    315.500000
5096    318.600006
5097    318.950012
5098    318.700012
Name: Close, Length: 5099, dtype: float64

In [3]:
df = df.dropna()
df.shape

(5085,)

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
tmp = scaler.fit(np.array(df).reshape(-1,1))
new_df = scaler.transform(np.array(df).reshape(-1,1))

 
print(new_df.shape)

(5085, 1)


In [ ]:
#obj = scaler.fit(np.array(df).reshape(-1,1))

#ori_df=scaler.inverse_transform(new_df)
#print(ori_df)

In [5]:
training_size=int(len(new_df)*0.67)
test_size=len(new_df)-training_size
train_data,test_data=new_df[0:training_size,:],new_df[training_size:len(new_df),:1]

print(train_data.shape)
print(test_data.shape)

(3406, 1)
(1679, 1)


In [6]:
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)

In [8]:
time_step=60
X_train, Y_train = create_dataset(train_data, time_step)
X_test, Y_test = create_dataset(test_data, time_step)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
print(X_train, Y_train)


(3345, 60) (3345,) (1618, 60) (1618,)
[[0.00996647 0.01149977 0.01402809 ... 0.00490983 0.00381695 0.00363752]
 [0.01149977 0.01402809 0.01497417 ... 0.00381695 0.00363752 0.0029198 ]
 [0.01402809 0.01497417 0.01443589 ... 0.00363752 0.0029198  0.00336022]
 ...
 [0.66995205 0.6788791  0.67930018 ... 0.78120321 0.77977152 0.79838356]
 [0.6788791  0.67930018 0.67854219 ... 0.77977152 0.79838356 0.79568859]
 [0.67930018 0.67854219 0.68536383 ... 0.79838356 0.79568859 0.79872041]] [0.0029198  0.00336022 0.00373539 ... 0.79568859 0.79872041 0.75880136]


In [ ]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
print(X_train.shape, X_test.shape)
print(X_train, Y_train)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

model=Sequential()
model.add(LSTM(64,return_sequences=True,input_shape=(60,1)))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

model.summary()

In [ ]:
model.fit(X_train,Y_train,validation_data=(X_test, Y_test),epochs=200,batch_size=32,verbose=1)

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
print(train_predict.shape, test_predict.shape)


In [ ]:
#obj = scaler.fit(np.array(new_df).reshape(-1,1))
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)



In [ ]:
import math
from sklearn.metrics import mean_squared_error
print(math.sqrt(mean_squared_error(Y_train,train_predict)))
print(math.sqrt(mean_squared_error(Y_test,test_predict)))

In [ ]:
tmp = scaler.fit(np.array(df).reshape(-1,1))
new_df = scaler.transform(np.array(df).reshape(-1,1))

In [ ]:
ori_df=scaler.inverse_transform(new_df)
print(ori_df)

In [ ]:
import matplotlib.pyplot as plt

look_back=60
trainPredictPlot = np.empty_like(new_df)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
# shift test predictions for plotting
testPredictPlot = np.empty_like(new_df)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(new_df)-1, :] = test_predict
# plot baseline and predictions
plt.plot(ori_df)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
print(1679-60)

here 1619 is the remaining days in the test data

total test data was 1679, if we take last 60 days data as the input for the forecast data

temp_input is the list of last 60 days data, which will be required as the input for forecasting data

In [ ]:
x_input=test_data[1619:].reshape(1,-1)
print(x_input.shape)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()


30 is the future days for which we'll get forecasted data

In [ ]:
from numpy import array

lst_output=[]
n_steps=60
i=0
while(i<30):
    
    if(len(temp_input)>60):
        print(len(temp_input))
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    

#print(lst_output)

In [ ]:
print(len(lst_output))
day_new=np.arange(1,61)
day_pred=np.arange(61,91)

In [ ]:
plt.plot(day_new,scaler.inverse_transform(new_df[5025:]))
plt.plot(day_pred,scaler.inverse_transform(lst_output))

In [ ]:
df1=new_df.tolist()
df1.extend(lst_output)
plt.plot(df1[1200:])


In [ ]:
df1=scaler.inverse_transform(df1).tolist()
plt.plot(df1)